In [9]:
%pip install -q --upgrade llama-index-llms-together llama-index llama-index-embeddings-jinaai llama-index-vector-stores-chroma docx2txt

### What is Retrieval Augmented Generation (RAG)?
RAG is a technique that enhances language models by combining them with a retrieval system. It allows the model to access and utilize external knowledge when generating responses.

The process typically involves:
#### Indexing a large corpus of documents

In [ ]:
import os
os.environ["TOGETHER_API_KEY"] = ""
os.environ["JINAAI_API_KEY"] = ""

### Call LLM

In [12]:
from llama_index.llms.together import TogetherLLM

llm = TogetherLLM(
    api_key=os.environ["TOGETHER_API_KEY"],
    model="meta-llama/Llama-Vision-Free",
)


llm_response = llm.complete("Tell me a joke")

print(llm_response)

Here's one:

What do you call a fake noodle?

An impasta.


### LLM Messages

In [13]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system", content="You are a pirate with a colorful personality"
    ),
    ChatMessage(role="user", content="What is your name"),
]
resp = llm.chat(messages)
print(resp)

assistant: Me hearty! Me name be Captain Blackbeak Betty, the most feared and infamous pirate to ever sail the Seven Seas! Me be a swashbucklin' scallywag with a heart o' gold and a spirit o' adventure. Me ship, the "Maverick's Revenge", be me home, and me trusty cutlass, "Betsy", be me best mate.

Now, what be bringin' ye to these fair waters? Are ye lookin' to join me crew and sail the seas in search o' treasure and glory? Or be ye just lookin' to test yer mettle against the greatest pirate to ever hoist the Jolly Roger? Either way, I be warnin' ye: I be a pirate o' me word, and I don't take kindly to landlubbers or scurvy dogs! Arrr!


In [14]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb

documents = SimpleDirectoryReader("/content/drive/MyDrive/Colab Notebooks/rag_lamaindex/docs/").load_data()

print(len(documents))



2


## Splitter

In [22]:
from llama_index.core import PromptHelper
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

Settings.node_parser = SentenceSplitter(chunk_size=300, chunk_overlap=30)

### Embeddings

In [20]:
from llama_index.embeddings.jinaai import JinaEmbedding

embed_model = JinaEmbedding(
    api_key=os.environ["JINAAI_API_KEY"],
    model="jina-embeddings-v3",
)

## Set Settings

In [24]:
Settings.llm = llm
Settings.embed_model = embed_model
Settings.num_output = 512
Settings.context_window = 4096
Settings.chunk_overlap_ratio = 0.1

### Create and persist Chroma vector store

In [25]:
db = chromadb.PersistentClient(path="/content/drive/MyDrive/Colab Notebooks/rag_lamaindex/chroma_db")
chroma_collection = db.get_or_create_collection("my_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    settings = Settings
    )

## Query Engine Urdu

In [29]:
query_engine = index.as_query_engine()
response = query_engine.query("نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟")
print(response)

2010 میں


## Chat Engine Urdu

In [30]:
query_engine = index.as_chat_engine()
response = query_engine.chat("نیکسٹ ویو ٹیکنالوجیز کی بنیاد کب رکھی گئی؟")
print(response)

نیکسٹ ویو ٹیکنالوجیز 2010 میں قائم ہوئی تھی۔


## Query Engine English

In [31]:
query_engine = index.as_query_engine()
response = query_engine.query("When was TechNova Solutions founded?")
print(response)

2015


## Chat Engine English

In [32]:
query_engine = index.as_chat_engine()
response = query_engine.chat("When was TechNova Solutions founded?")
print(response)

TechNova Solutions was founded in 2015.


In [33]:
query_engine = index.as_chat_engine()
response = query_engine.chat("What is the website of TechNova Solutions?")
print(response)

The website of TechNova Solutions is www.technovasolutions.com.
